# Calculating the Friction Surface for the Isle of Man

In [ ]:
import geopyspark as gps
import numpy as np
import pyproj
import fiona

from functools import partial
from shapely.geometry import shape, MultiPoint, MultiLineString
from shapely.ops import transform
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from geonotebook.wrappers import VectorData, TMSRasterData

conf = gps.geopyspark_conf(appName="gps-osm-ingest", master='local[*]')
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
conf.set('spark.ui.enabled', True)
conf.set('spark.default.parallelism', 8)
conf.set('spark.master.memory', '9500M')

sc = SparkContext(conf=conf)
pysc = gps.get_spark_context()
session = SparkSession.builder.config(conf=pysc.getConf()).enableHiveSupport().getOrCreate()

In [ ]:
M.set_center(-4.5208740234375, 54.23714281711491, 9)

## Calculating the Friction Layer From OSM and SRTM Data

In [ ]:
# Download the orc file from S3

!curl -o /tmp/isle-of-man.orc https://s3.amazonaws.com/geotrellis-test/eac/orc/isle-of-man.orc

In [ ]:
# This cell contains the logic that assigns each section of road a
# speed based on the type of road that section is.

default_speeds = {
'motorway':100,
'trunk':70,
'primary':65,
'secondary':60,
'tertiary':50,
'unclassified':30,
'residential':30,
'service':20,
'motorway_link':70,
'trunk_link':65,
'primary_link':60,
'secondary_link':50,
'tertiary_link':40,
'living_street':10,
'pedestrian':6,
'track':20,
'road':35}

country_speeds = {
'at:urban':50,
'at:rural':100,
'at:trunk':100,
'at:motorway':130,
'ch:urban':50,
'ch:rural':80,
'ch:trunk':100,
'ch:motorway':120,
'cz:urban':50,
'cz:rural':90,
'cz:trunk':130,
'cz:motorway':130,
'dk:urban':50,
'dk:rural':80,
'dk:motorway':130,
'de:living_street':7,
'de:urban':50,
'de:walk':7,
'de:rural':100,
'fi:urban':50,
'fi:rural':80,
'fi:trunk':100,
'fi:motorway':120,
'fr:urban':50,
'fr:rural':90,
'fr:trunk':110,
'fr:motorway':130,
'hu:urban':50,
'hu:rural':90,
'hu:trunk':110,
'hu:motorway':130,
'it:urban':50,
'it:rural':90,
'it:trunk':110,
'it:motorway':130,
'jp:national':60,
'jp:motorway':100,
'ro:urban':50,
'ro:rural':90,
'ro:trunk':100,
'ro:motorway':130,
'ru:living_street':20,
'ru:rural':90,
'ru:urban':60,
'ru:motorway':110,
'sk:urban':50,
'sk:rural':90,
'sk:trunk':130,
'sk:motorway':130,
'si:urban':50,
'si:rural':90,
'si:trunk':110,
'si:motorway':130,
'se:urban':50,
'se:rural':70,
'se:trunk':90,
'se:motorway':110,
'gb:nsl_single':96.54,
'gb:nsl_dual':112.63,
'gb:motorway':112.63,
'ua:urban':60,
'ua:rural':90,
'ua:trunk':110,
'ua:motorway':130,
'living_street':6}

words = ['maxspeed', 'ambiguous', 'signals', 
         'none', 'walk', 'variable', 
         'national', 'fixme', 'unposted', 'implicit']

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def default_speed(highway):
    if not highway in default_speeds:
        return default_speeds['road']
    else:
        return default_speeds[highway]

def get_maxspeed(speed, units, highway):
    speeds = speed.split(';|,-')
    maxspeed = 0
    for sp in speeds:
        sp = sp.replace(units, '')
        if (is_number(sp)):
            if units == 'mph':
                sp = 1.609 * float(sp) 
            elif units == 'knots':
                sp = 1.852 * float(knots)
            else:
                sp = float(sp)
                
            if sp > maxspeed:
                maxspeed = sp
    if maxspeed > 0:
        speed = maxspeed
    else:
        speed = default_speed(highway)

    return speed

def get_highway_cellvalue(osm_feature):   
    highway = osm_feature.properties.tags['highway']
    speed = osm_feature.properties.tags.get('maxspeed', '')
                                
    speed = speed.lower().strip()
        
    # if we don't have a speed, give it a default
    if len(speed) == 0:
        speed = default_speed(highway)
    elif not is_number(speed):
        if 'kph' in speed:
            speed = get_maxspeed(speed, 'kph', highway)
        elif 'km/h' in speed:
            speed = get_maxspeed(speed, 'km/h', highway)
        elif 'kmh' in speed:
            speed = get_maxspeed(speed, 'kmh', highway)
        elif 'mph' in speed:
            speed = get_maxspeed(speed, 'mph', highway)
        elif 'knots' in speed:
            speed = get_maxspeed(speed, 'knots', highway)
        elif speed in country_speeds:
            speed = country_speeds[speed]
        elif speed in words:
            speed = default_speed(highway)
        else:
            speed = get_maxspeed(speed, '', highway)            
    if float(speed) <= 0.0:
        speed = default_speed(highway)

    speed = float(speed)
    return gps.CellValue(speed, speed)

### Reading and Formatting the OSM Data

In [ ]:
# Reading in the ORC file as a Spark DataFrame

file_uri = "file:///tmp/isle-of-man.orc"
osm_dataframe = session.read.orc(file_uri)

In [ ]:
# Get all of the lines that are contained within the DataFrame

osm = gps.vector_pipe.osm_reader.from_dataframe(osm_dataframe)
lines = osm.get_line_features_rdd()

In [ ]:
# Only highways are of interest
highways = lines.filter(lambda feature: 'highway' in feature.properties.tags)

# Shows the OSM tags of the first element in the highways RDD
highways.take(1)[0].properties.tags

In [ ]:
# Encode highway speeds as feature properties for rasterization
highway_features = highways.map(
    lambda feature:    
        gps.Feature(feature.geometry, get_highway_cellvalue(feature)))

In [ ]:
# The first Feature in the RDD that contains the geometry and CellValue
highway_features.take(1)

In [ ]:
highway_raster = gps.geotrellis.rasterize_features(
    features = highway_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    partition_strategy=gps.SpatialPartitionStrategy(16)
).persist(StorageLevel.MEMORY_AND_DISK)

### Displaying the Rasterized Highways

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(8, 100, 4), 
    color_list = gps.get_colors_from_matplotlib('magma'))

osm_wm = highway_raster.tile_to_layout(
    gps.GlobalLayout(tile_size=256),
    target_crs=3857
)

layer = gps.TMS.build(osm_wm.convert_data_type(gps.CellType.INT8, 0).pyramid(), color_map)
M.add_layer(TMSRasterData(layer), name="OSM-highways")

In [ ]:
M.remove_layer(M.layers[0])

### Reading the SRTM Data and Calculating the Friction Layer

In [ ]:
isle_extent = gps.Extent(xmin=-4.921879863281248, ymin=53.964837753906245, 
                         xmax=-4.218754882812476, ymax=54.4921814453125)

# Read SRTM for the same extent as rasterized OSM features
srtm = gps.query(
    uri="s3://geotrellis-test/dg-srtm",
    layer_name="srtm-wsg84-gps", 
    layer_zoom = 0,
    query_geom = isle_extent,
    query_proj = 4326,
    num_partitions = 16
)

# Tile SRTM layer to same layout as rasterized OSM features
tiled_srtm = srtm.tile_to_layout(highway_raster.layer_metadata,
                                 partition_strategy=gps.SpatialPartitionStrategy()
                                ).convert_data_type(gps.CellType.INT16, 0)

In [ ]:
# Calculate the ZFactors based on the SRTM Layer's projection and units
zfactor = gps.geotrellis.zfactor_lat_lng_calculator('Meters')

# Perform the Tobler operation on the resulting slope layer
tobler_raster = tiled_srtm.slope(zfactor).tobler()

# Create the friction layer by performing a local max on the Tobler layer
# using the Highway layer
friction = tobler_raster.local_max(highway_raster)

In [ ]:
reprojected = friction.tile_to_layout(
    target_crs = 3857,
    layout = gps.GlobalLayout(tile_size=256),
    resample_method = gps.ResampleMethod.MAX
).convert_data_type(gps.CellType.FLOAT32, -2147483648.0).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
pyramid = reprojected.pyramid().persist(StorageLevel.MEMORY_AND_DISK_SER)

### Displaying the Friction Layer

In [ ]:
# Building the color map from the histogram of the pyramid
hist_color_map = gps.ColorMap.build(pyramid.get_histogram(), 'magma')
hist_layer = gps.TMS.build(pyramid, hist_color_map)

M.add_layer(TMSRasterData(hist_layer), name="ToblerOSM-from-hist")

In [ ]:
M.remove_layer(M.layers[0])

## Calculating Cost Distance Using the Tobler Layer

### Reading and Formatting the Population Centers

In [ ]:
# Downloads the Manx population centers from S3

!curl -o /tmp/manx_pop_centers.geojson https://s3.amazonaws.com/geotrellis-test/eac/orc/manx_pop_centers.geojson

In [ ]:
with fiona.open("/tmp/manx_pop_centers.geojson") as source:
    pop_centers = [shape(f['geometry']) for f in source]

In [ ]:
# The population centers in question
MultiPoint(pop_centers)

In [ ]:
# Reproject the population centers so that they are in WebMercator
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

reprojected_pop_centers = [transform(project, geom) for geom in pop_centers]

### Calculating Cost Distance

In [ ]:
cost_distance = gps.cost_distance(3.6 / reprojected,
                                  reprojected_pop_centers,
                                  150000.0)

In [ ]:
cd_pyramid = cost_distance.pyramid().persist(StorageLevel.MEMORY_AND_DISK_SER)

### Displaying Cost Distance and the Population Centers

In [ ]:
cd_color_map = gps.ColorMap.build(cd_pyramid.get_histogram(), 'viridis')
cd_layer = gps.TMS.build(cd_pyramid, cd_color_map)

M.add_layer(TMSRasterData(cd_layer), name="ToblerOSM-cost-distance")

In [ ]:
M.add_layer(VectorData("/tmp/manx_pop_centers.geojson"),
            name="Manx Population Centers",
            colors=[0xff0000])

In [ ]:
x = 0
while x < len(M.layers):
    M.remove_layer(M.layers[x])
    x += 1

## Saving and Cleanup

In [ ]:
# Write the firction layer pyramid to the catalog
for layer in pyramid.levels.values():
    gps.write("s3://geotrellis-test/dg-osm-test-man", "gps-osm-ingest", layer)

In [ ]:
highway_raster.unpersist()
reprojected.unpersist()
pyramid.unpersist()
cd_pyramid.unpersist()